In [1]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

A new version of galpy (1.9.2) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [2]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
#dfv = dfv.loc[dfv['extinction'].between(0.0003, 1)]
dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 2
scatter_edenhofer = go.Scatter3d(
    x=dfv['X'].values[::ds_index],
    y=dfv['Y'].values[::ds_index],
    z=dfv['Z'].values[::ds_index],
    mode='markers',
    marker=dict(size=5.,
                color='gray',
                # color=dfv['extinction'].values[::ds_index],
                # colorscale='Gray_r',
                cmin = 0.0007,
                cmax = 0.005,
                symbol='circle',
                opacity=.01),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0


hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0

In [3]:
df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_catalog_oct_13.csv')
df_hunt_full = df_hunt_full.rename(columns={'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'})

df_hunt_good = df_hunt_full.loc[
    (df_hunt_full['n_rv_gaia_apogee_galah'] >= 2) & 
    (df_hunt_full['U_err'] < 10) & 
    (df_hunt_full['V_err'] < 10) & 
    (df_hunt_full['W_err'] < 10)
    ]
df_hunt_young = df_hunt_good.loc[df_hunt_good['age_myr'] < 15]
df_hunt_middle_aged = df_hunt_good.loc[df_hunt_good['age_myr'].between(15, 30)]
df_hunt_older = df_hunt_good.loc[df_hunt_good['age_myr'].between(0, 30)]

In [4]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']
gamma_vel = df_hunt_gagne[df_hunt_gagne['family'] == 'gamma_velorum']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [0.], 'n_stars' : [1]})


In [5]:
grouping_dict = {
    'Cluster families' : ['⍺Per', 'M6', 'Cr135', '𝝲Vel', 'Sun'],
    'Younger clusters' : ['Young clusters', 'Older clusters'],
    'Formation of Gamma Vel' : ['M6', 'Cr135', '𝝲Vel']
}

# Dark plot

In [6]:
ap_data = StarClusterData(ap, data_name = '⍺Per', min_size = 0, max_size = 7, color = 'violet', opacity = 1., marker_style = 'circle')
m6_data = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 7, color = 'cyan', opacity = 1., marker_style = 'circle')
cr135_data = StarClusterData(cr135, data_name = 'Cr135', min_size = 0, max_size = 7, color = 'orange', opacity = 1., marker_style = 'circle')
gamma_vel_data = StarClusterData(gamma_vel, data_name = '𝝲Vel', min_size = 0, max_size = 7, color = 'red', opacity = 1., marker_style = 'circle')
young_data = StarClusterData(df_hunt_young, data_name = 'Young clusters', min_size = 0., max_size = 7, color = 'black', opacity = 1., marker_style = 'circle')
older_data = StarClusterData(df_hunt_older, data_name = 'Older clusters', min_size = 0., max_size = 7, color = 'black', opacity = 1., marker_style = 'circle')
sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 7, color = 'yellow', opacity = 1., marker_style = 'circle')

sc = StarClusterCollection([young_data, older_data, sun_data, ap_data, m6_data, cr135_data, gamma_vel_data])

xyz_widths = (2000, 2000, 300)
plot_3d = StarClusters3DPlotter(data_collection = sc, xyz_widths = xyz_widths, figure_theme = 'gray', trace_grouping_dict=grouping_dict)
scatter_edenhofer['marker']['color'] = 'white'
scatter_edenhofer['marker']['opacity'] = 0.008

save_name = '/Users/cam/Downloads/gal_movie.html'
#save_name = '/Users/cam/Downloads/gamma_vel_movie.html'
time_int = np.round(np.arange(0., -60.5, -0.5), 1)
fig3d_m6 = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = static_traces, 
    static_traces_times = static_traces_times, 
    focus_group = None,
    fade_in_time = 20 # Myr,
)


# Noir plot

In [7]:
# ap_data = StarClusterData(ap, data_name = '⍺Per', min_size = 0, max_size = 8, color = 'black', opacity = 0.4, marker_style = 'diamond')
# m6_data = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 8, color = 'black', opacity = 1., marker_style = 'x')
# cr135_data = StarClusterData(cr135, data_name = 'Cr135', min_size = 0, max_size = 8, color = 'black', opacity = 0.4, marker_style = 'cross')
# young_data = StarClusterData(df_hunt_young, data_name = 'Young clusters', min_size = 0, max_size = 8, color = 'black', opacity = 0.4, marker_style = 'circle')
# sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 5, color = 'black', opacity = 1., marker_style = 'circle')
# sc_m6 = StarClusterCollection([young_data, ap_data, m6_data, cr135_data, sun_data])

# plot_3d = StarClusters3DPlotter(data_collection = sc_m6, xyz_widths = (600, 600, 300), figure_theme = 'light', plotly_light_template='ggplot2')
# #scatter_lb['marker']['color'] = 'black'

# save_name = '/Users/cam/Downloads/gal_movie_noir.html'
# time_int = np.round(np.arange(0., -60.5, -0.5), 1)
# fig3d_m6 = plot_3d.generate_3d_plot(
#     time = time_int,
#     show = False, 
#     save_name =save_name, 
#     static_traces = static_traces, 
#     static_traces_times = static_traces_times, 
#     focus_group = 'M6',
#     fade_in_time = 15 # Myr
# )

# Solarized

In [8]:
# ap_data = StarClusterData(ap, data_name = '⍺Per', min_size = 0, max_size = 8, color = 'violet', opacity = 1., marker_style = 'circle')
# m6_data = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 8, color = 'cornflowerblue', opacity = 1., marker_style = 'circle')
# cr135_data = StarClusterData(cr135, data_name = 'Cr135', min_size = 0, max_size = 8, color = 'orange', opacity = 1., marker_style = 'circle')
# young_data = StarClusterData(df_hunt_young, data_name = 'Young clusters', min_size = 0, max_size = 8, color = 'black', opacity = 1., marker_style = 'circle')
# sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 5, color = 'yellow', opacity = 1., marker_style = 'circle')
# sc_m6 = StarClusterCollection([young_data, ap_data, m6_data, cr135_data, sun_data])

# plot_3d = StarClusters3DPlotter(data_collection = sc_m6, xyz_widths = (600, 600, 300), figure_theme = 'solarized_light')
# scatter_lb['marker']['color'] = 'black'

# save_name = '/Users/cam/Downloads/gal_movie_solzarized.html'
# time_int = np.round(np.arange(0., -60.5, -0.5), 1)
# fig3d_m6 = plot_3d.generate_3d_plot(
#     time = time_int,
#     show = False, 
#     save_name =save_name, 
#     static_traces = static_traces, 
#     static_traces_times = static_traces_times, 
#     fade_in_time = 15 # Myr
# )